In [57]:
from samalg import SAM
from samalg.gui import SAMGUI
import samalg.utilities as ut

import scanpy.external as sce
import scanpy as sc
import pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import sparse, io
from scipy import stats

In [ ]:
adata = sc.read_h5ad('adata_xxx.h5ad')

In [ ]:
#self defined functions

def generate_disdata(adata_HB2):

    obs_HB2 = adata_HB2.X
    obs_HB2 = obs_HB2.toarray()
    obs_HB2 = pd.DataFrame(obs_HB2,columns=adata_HB2.var.index)

    row_num=pd.DataFrame(adata_HB2.obs['array_row'])
    col_num=pd.DataFrame(adata_HB2.obs['array_col'])

    row_num.index=obs_HB2.index
    col_num.index=obs_HB2.index

    obs_HB2=obs_HB2.merge(row_num,left_index=True,right_index=True)
    obs_HB2=obs_HB2.merge(col_num,left_index=True,right_index=True)

    obs_HB2['array_col']=obs_HB2['array_col']/1.732
    
    return obs_HB2;

def distance_to_line (x,y,x1,y1,x2,y2):
  d=((y-y1)-(y2-y1)*(x1-x)/(x1-x2))*(x1-x2)/((x2-x1)*(x2-x1)+(y2-y1)*(y2-y1))**0.5
  d=-d
  return d;

def express_sampling_new (dis,expre):
    
    distance=np.linspace(-5,11,21)
    ave_expre=np.zeros(len(distance))
    sample_number=np.zeros(len(distance))

    for i in range(len(dis)):
        for j in range(len(distance)-1):
            if dis[i]>distance[j] and dis[i]<distance[j+1]:
                ave_expre[j]=ave_expre[j]+expre[i]
                sample_number[j]=sample_number[j]+1
    
    ave_expre=ave_expre/sample_number
    ave_expre[np.isnan(ave_expre)]=0
    
    return distance, ave_expre;

def main_express_new (x1,y1,x2,y2,x3,y3,range_th_l,range_th_r,gene_id,obs_HB2):
    dis=[]
    expre=[]
    for i in range(len(obs_HB2)):
      if obs_HB2.iloc[i]['array_col']>x2 and obs_HB2.iloc[i]['array_col']<x1:
          d=distance_to_line (obs_HB2.iloc[i]['array_col'],obs_HB2.iloc[i]['array_row'],x1,y1,x2,y2)
          if d>range_th_l and d<range_th_r:
              dis.append(d)
              expre.append(obs_HB2.iloc[i][gene_id])
    
      if obs_HB2.iloc[i]['array_col']>x3 and obs_HB2.iloc[i]['array_col']<x2:
          d=distance_to_line (obs_HB2.iloc[i]['array_col'],obs_HB2.iloc[i]['array_row'],x2,y2,x3,y3)
          if d>range_th_l and d<range_th_r:
              dis.append(d)
              expre.append(obs_HB2.iloc[i][gene_id])
    
    return dis, expre;

In [ ]:
#calculate P value and fold change

geneid=adata_HB2_norm.var.index
obs = adata_HB2[:,geneid].X
obs = obs.toarray()
obs = pd.DataFrame(obs,columns=geneid,index=adata_HB2.obs['identity'])

p_value=np.zeros(len(geneid))
fold=np.zeros(len(geneid))

for i in range(len(geneid)):
    gene_id=geneid[i]
    [noting,p]=stats.mannwhitneyu(obs.loc['dendrite',gene_id],obs.loc['soma',gene_id])
    p_value[i]=p

    if (np.mean(obs.loc['dendrite',gene_id])>0) & (np.mean(obs.loc['soma',gene_id])>0):
        fold[i]=np.log2(np.mean(obs.loc['dendrite',gene_id])/np.mean(obs.loc['soma',gene_id]))
        
d={'p_value':p_value, 'fold_change':fold, 'mark': mark}
dendrite_gene=pd.DataFrame(data=d,index=geneid)

In [ ]:
#generate array data

obs=generate_disdata

In [ ]:
#parameters set

x1=57*1.155
y1=45

x2=43*1.155
y2=57.5

x3=27*1.155
y3=58

range_l=-5
range_r=11

In [ ]:
#label soma region & plot

plt.figure(figsize=(6,5))
plt.scatter(obs['array_col'], obs['array_row'], c=obs['gene_name']/np.max(obs['gene_name']), s=3)

plt.scatter(x1, y1, c='k', s=5)
plt.scatter(x2, y2, c='k', s=5)
plt.scatter(x3, y3, c='k', s=5)

plt.plot([x1,x2], [y1,y2], c='k')
plt.plot([x2,x3], [y2,y3], c='k')

plt.colorbar()
plt.show()

In [ ]:
#linear profile plot

gene_id='Lpl' #as an example

dis, expre=main_express_new (x1,y1,x2,y2,x3,y3,range_l,range_r,gene_id,ob2_HB2_raw)
distance, ave_expre = express_sampling_new (dis,expre)

plt.figure(figsize=(3.5,3))
plt.plot(distance, ave_expre)
plt.show()